This Jupyter NB has all of the data preprocessing and modeling for the Team of the Season Predictor. In the Notebook, I will go through data preprocessing, various ML arcitectures that might work for this Classification problem, stastical comparisons between the models, and a final conclusion on what model I will use and why I chose it. Some more insight into the data can be found in WebScrape Juptyer NB found in the Practice folder.

# Data Preprocessing

The first part of this code is from `Webscrape.ipynb`in the Practice folder. 

In [1]:
"""IMPORTS"""
# Data Preprocessing
import pandas as pd


# Web Scraping
import requests
from bs4 import BeautifulSoup
from io import StringIO
import re

In [2]:
""" WEBSCRAPING PLAYER STATS """

# Player Stats from FBREF
url_2023_2024 = 'https://fbref.com/en/comps/9/stats/Premier-League-Stats'
url_2022_2023 = 'https://fbref.com/en/comps/9/2022-2023/stats/2022-2023-Premier-League-Stats'
url_2021_2022 = 'https://fbref.com/en/comps/9/2021-2022/stats/2021-2022-Premier-League-Stats'
url_2020_2021 = 'https://fbref.com/en/comps/9/2020-2021/stats/2020-2021-Premier-League-Stats'
url_2019_2020 = 'https://fbref.com/en/comps/9/2019-2020/stats/2019-2020-Premier-League-Stats'
url_2018_2019 = 'https://fbref.com/en/comps/9/2018-2019/stats/2018-2019-Premier-League-Stats'
url_2017_2018 = 'https://fbref.com/en/comps/9/2017-2018/stats/2017-2018-Premier-League-Stats'

urls = [ url_2023_2024 , url_2022_2023 , url_2021_2022 , url_2020_2021 , url_2019_2020, url_2018_2019, url_2017_2018]
player_stat_dfs = []


for link in urls:
    temp = pd.read_html(
        requests.get(link).text.replace('<!--','').replace('-->','')
        ,attrs={'id':'stats_standard'}
    )[0]
    
    temp_data = pd.DataFrame(temp)
    
    player_stat_dfs.append(temp_data)
    


"""PULLING TEAM OF THE SEASON INFO"""
"""TARGET SEASONS: 2017-2018 - 2023-2024"""
from bs4 import BeautifulSoup
from io import StringIO


# Get URLS for TOTS info
url_tots_wiki_2020s = 'https://en.wikipedia.org/wiki/PFA_Team_of_the_Year_(2020s)'
url_tots_wiki_2010s = 'https://en.wikipedia.org/wiki/PFA_Team_of_the_Year_(2010s)'

tots_urls = [url_tots_wiki_2020s , url_tots_wiki_2010s]
tots_dfs = []

for url in tots_urls:
    
    # Send an HTTP GET request to the URL
    temp_response = requests.get(url)
    
    if temp_response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(temp_response.content, 'html.parser')
        
        # Find Where all the PL Headers are 
        h4_headers = soup.find_all('h4')
        
        # Get PL Tags
        premier_league_tags = [tag for tag in h4_headers if 'Premier League' in tag.text]
        
        
        
        for header in premier_league_tags:
            # Find the next sibling table tag
            table = header.find_next_sibling('table')
            
            # If a table is found, print it
            if table:
                df = pd.read_html(StringIO(str(table)))[0]
                tots_dfs.append(df) 
    
    else:
        print("Failed to retrieve the page. Status code:", temp_response.status_code)

/var/folders/_x/czhgmpkn0lx4yd57n3w9bwv00000gn/T/ipykernel_14717/528283739.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp = pd.read_html(
/var/folders/_x/czhgmpkn0lx4yd57n3w9bwv00000gn/T/ipykernel_14717/528283739.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp = pd.read_html(
/var/folders/_x/czhgmpkn0lx4yd57n3w9bwv00000gn/T/ipykernel_14717/528283739.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp = pd.read_html(
/var/folders/_x/czhgmpkn0lx4yd57n3w9bwv00000gn/T/ipykernel_14717/528283739.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

In [3]:
"""PUT TABLES INTO DICTIONARY BY YEAR"""

yr_playerStat_dict = {
    '2023-2024' : player_stat_dfs[0],
    '2022-2023' : player_stat_dfs[1],
    '2021-2022' : player_stat_dfs[2],
    '2020-2021' : player_stat_dfs[3],
    '2019-2020' : player_stat_dfs[4],
    '2018-2019' : player_stat_dfs[5],
    '2017-2018' : player_stat_dfs[6]
}

"""MAPPING TABLES TO APROPRIATE YEAR IN DICTIONARY (year -> TOTS Table)"""

yr_totsNoms_dict = {
    '2019-2020' : tots_dfs[0] ,
    '2020-2021' : tots_dfs[1] ,
    '2021-2022' : tots_dfs[2] ,
    '2022-2023' : tots_dfs[3] , 
    '2017-2018' : tots_dfs[-2] , 
    '2018-2019' : tots_dfs[-1] , 
}

In [4]:
list_seasons = ['2017-2018' , '2018-2019' , '2019-2020' , '2020-2021', '2021-2022' , '2022-2023'] # Exclude current Season

# All DataFrames Stored
merged_dataframes = []

for szn in list_seasons:
    tmp_tots_stats = yr_totsNoms_dict[szn]
    tmp_plyr_stats = yr_playerStat_dict[szn]
    
    # Strip Player name to get rid of special chars
    tmp_tots_stats['Player'] = tmp_tots_stats['Player'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x).strip())
    
    # Updating Names for TOTS Nominees Table to match Player Stats DF
    tmp_tots_stats = tmp_tots_stats.rename(columns={'App.': 'Tots_Apps' , 'Pos.': 'Pos' , 'Club': 'Squad'})
    
    # Drop Upper Level Layers
    tmp_plyr_stats.columns = tmp_plyr_stats.columns.droplevel([0])
    
    # Merge the DataFrames
    # Join tables, keep all of info from player stats , join on Player name and squad
    tmp_df_merged = pd.merge(tmp_plyr_stats, tmp_tots_stats, on=['Player', 'Squad'], how='left')
    
    #Fix Pos_x 
    tmp_df_merged['Pos'] = tmp_df_merged['Pos_x'].str.split(',').str[0]
    
    # If Tots_Apps = NaN
    # Make that = 0
    tmp_df_merged['Tots_Apps'] = tmp_df_merged['Tots_Apps'].fillna(0)
    
    # Nation 
    # Use the ALL CAPS Countries
    tmp_df_merged['Nation'] = tmp_df_merged['Nation'].str.split().str[-1]
    
    # Drop Matches, Rk , Extra Pos's
    tmp_df_merged = tmp_df_merged.drop(columns=['Rk', 'Matches' , 'Pos_x' , 'Pos_y' ])
    
    merged_dataframes.append(tmp_df_merged)
    
    
for mdf in merged_dataframes: mdf.drop(mdf[mdf['Player'] == 'Player'].index, inplace=True)
    



In [5]:
print(len(merged_dataframes))
merged_dataframes[0]

6


,Player,Nation,Squad,Age,Born,MP,Starts,Min,90s,Gls,...,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Tots_Apps,Pos
0,Patrick van Aanholt,NED,Crystal Palace,26,1990,28,25,2184,24.3,5,...,0.25,0.21,0.25,0.13,0.09,0.21,0.13,0.21,0.0,DF
1,Rolando Aarons,ENG,Newcastle Utd,21,1995,4,1,139,1.5,0,...,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.04,0.0,MF
2,Tammy Abraham,ENG,Swansea City,19,1997,31,15,1726,19.2,5,...,0.31,0.26,0.31,0.35,0.08,0.44,0.35,0.44,0.0,FW
3,Charlie Adam,SCO,Stoke City,31,1985,11,5,411,4.6,0,...,0.00,0.00,0.00,0.36,0.27,0.63,0.19,0.46,0.0,MF
4,Adrián,ESP,West Ham,30,1987,19,19,1710,19.0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,GK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,Wilfried Zaha,CIV,Crystal Palace,24,1992,29,28,2549,28.3,9,...,0.42,0.32,0.42,0.33,0.17,0.49,0.33,0.49,0.0,FW
546,Davide Zappacosta,ITA,Chelsea,25,1992,22,12,1098,12.2,1,...,0.16,0.08,0.16,0.11,0.11,0.22,0.11,0.22,0.0,DF
547,Marvin Zeegelaar,NED,Watford,26,1990,12,12,994,11.0,0,...,0.09,0.00,0.09,0.00,0.09,0.09,0.00,0.09,0.0,DF
548,Oleksandr Zinchenko,UKR,Manchester City,20,1996,8,6,532,5.9,0,...,0.00,0.00,0.00,0.02,0.07,0.09,0.02,0.09,0.0,DF


#### Cleaning the Data

In [6]:
# Add in the target column
for mdf in merged_dataframes:
    mdf['TOTS'] = mdf['Tots_Apps'].apply(lambda x: 1 if x > 0 else 0)

In [8]:
merged_dataframes[0].head()

,Player,Nation,Squad,Age,Born,MP,Starts,Min,90s,Gls,...,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Tots_Apps,Pos,TOTS
0,Patrick van Aanholt,NED,Crystal Palace,26,1990,28,25,2184,24.3,5,...,0.21,0.25,0.13,0.09,0.21,0.13,0.21,0.0,DF,0
1,Rolando Aarons,ENG,Newcastle Utd,21,1995,4,1,139,1.5,0,...,0.00,0.00,0.04,0.00,0.04,0.04,0.04,0.0,MF,0
2,Tammy Abraham,ENG,Swansea City,19,1997,31,15,1726,19.2,5,...,0.26,0.31,0.35,0.08,0.44,0.35,0.44,0.0,FW,0
3,Charlie Adam,SCO,Stoke City,31,1985,11,5,411,4.6,0,...,0.00,0.00,0.36,0.27,0.63,0.19,0.46,0.0,MF,0
4,Adrián,ESP,West Ham,30,1987,19,19,1710,19.0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,GK,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Nathaniel Clyne,ENG,Liverpool,26,1991,3,2,174,1.9,0,...,0.00,0.00,0.07,0.04,0.10,0.07,0.10,0.0,DF,0
99,Séamus Coleman,IRL,Everton,28,1988,12,12,1035,11.5,0,...,0.00,0.09,0.09,0.06,0.15,0.09,0.15,0.0,DF,0
100,James Collins,WAL,West Ham,33,1983,13,12,1050,11.7,1,...,0.09,0.09,0.03,0.02,0.05,0.03,0.05,0.0,DF,0
101,Lewis Cook,ENG,Bournemouth,20,1997,29,25,2237,24.9,0,...,0.00,0.08,0.02,0.06,0.08,0.02,0.08,0.0,MF,0
